In [25]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os
import math



In [26]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

In [27]:
raw_file="Raw data 26-April'25 According to 4_7.5.csv"
gst_file="GST 26-April'25 According to 4_7.5.csv"
tracker_file="EDD Tracker till 26th April'25.csv"
format_file="edd_format.csv"
ops_file="ops_status.csv"
old_tracker_file="old_tracker.csv"


In [28]:
raw_data= pd.read_csv(raw_file,memory_map=True)
gst_data=pd.read_csv(gst_file,memory_map=True)
tracker_data=pd.read_csv(tracker_file,memory_map=True)
master_format=pd.read_csv(format_file,memory_map=True)
ops_status=pd.read_csv(ops_file,memory_map=True)
old_tracker=pd.read_csv(old_tracker_file,memory_map=True)

In [29]:
raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,sellerid,bank_response,bank_name,pan_no_business,pan_no_personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,84152466,wGkLWY00056042674942,wGkLWY00056042674942,84152466,2024-06-11 13:33:56.000,2024-06-11 13:39:34.000,1.899359e+09,DIY,QR,PTY,11684152466,"{""sellerId"":""11684152466""}",PTYES,CHVPG4162K,CHVPG4162K,NaN,No,Food,Juice and Snacks,5411,PROPRIETORSHIP,46255.0,400000,1595.0,49445.0,1,1,0.115638,Not_required,Pending for share,100000.0
1,87831302,wIoyDM46851761551383,wIoyDM46851761551383,87831302,2025-04-21 16:16:44.000,2025-04-21 16:17:17.000,7.043959e+06,Transit Offline,EDC,PTY,18987831302,"{""sellerId"":""18987831302""}",PTYES,CXIPS8895N,CXIPS8895N,33CXIPS8895N1Z9,Yes,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN
2,85538028,vmiSgy43648835767287,vmiSgy43648835767287,85538028,2024-10-26 12:57:41.000,2024-10-26 12:58:49.000,1.745045e+09,Offline Unorganized,EDC,PTY,14085538028,"{""sellerId"":""14085538028""}",PTYES,AHGPR9408N,AHGPR9408N,10AHGPR9408N1Z0,Yes,Beauty and Wellness,Salon and Parlour,7230,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [30]:
gst_data=gst_data.rename(columns={'_col3':'updated_at'})
gst_data.head(3)

,solution_value,cust_id,solution_value.1,updated_at
0,mCJLRw40248294050664,1.950030e+09,Slab: Rs. 40 lakhs to 1.5 Cr.,2025-04-26
1,EcuCam75972053723310,1.278551e+09,Slab: Rs. 0 to 40 lakhs,2025-04-26
2,nRccGs29005105680621,5.203776e+07,Slab: Rs. 0 to 40 lakhs,2025-04-25


In [31]:
# Convert the two columns to datetime first
raw_data['mid_creation_date'] = pd.to_datetime(raw_data['mid_creation_date'], errors='coerce')
raw_data['channel_date'] = pd.to_datetime(raw_data['channel_date'], errors='coerce')

# Then format them to 'dd-mm-yyyy' string format
raw_data['mid_creation_date'] = raw_data['mid_creation_date'].dt.strftime('%d-%m-%Y')
raw_data['channel_date'] = raw_data['channel_date'].dt.strftime('%d-%m-%Y')

raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,sellerid,bank_response,bank_name,pan_no_business,pan_no_personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,84152466,wGkLWY00056042674942,wGkLWY00056042674942,84152466,11-06-2024,11-06-2024,1.899359e+09,DIY,QR,PTY,11684152466,"{""sellerId"":""11684152466""}",PTYES,CHVPG4162K,CHVPG4162K,NaN,No,Food,Juice and Snacks,5411,PROPRIETORSHIP,46255.0,400000,1595.0,49445.0,1,1,0.115638,Not_required,Pending for share,100000.0
1,87831302,wIoyDM46851761551383,wIoyDM46851761551383,87831302,21-04-2025,21-04-2025,7.043959e+06,Transit Offline,EDC,PTY,18987831302,"{""sellerId"":""18987831302""}",PTYES,CXIPS8895N,CXIPS8895N,33CXIPS8895N1Z9,Yes,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN
2,85538028,vmiSgy43648835767287,vmiSgy43648835767287,85538028,26-10-2024,26-10-2024,1.745045e+09,Offline Unorganized,EDC,PTY,14085538028,"{""sellerId"":""14085538028""}",PTYES,AHGPR9408N,AHGPR9408N,10AHGPR9408N1Z0,Yes,Beauty and Wellness,Salon and Parlour,7230,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [32]:
tracker_data=tracker_data.fillna(0)
tracker_data.isna().sum()

Bank Response    0
Final Remarks    0
dtype: int64

In [33]:
len(ops_status)+len(tracker_data)+len(old_tracker)

316948

In [34]:
tracker_data=tracker_data.rename(columns={'Bank Response':'bank_response','Final Remarks':'final_remarks'})
old_tracker=old_tracker.rename(columns={'bank_response':'bank_response','Final Remarks':'final_remarks'})
ops_status=ops_status.rename(columns={'sellerid':'bank_response','OPS status':'final_remarks'})



ops_status_data= pd.concat([ops_status, tracker_data,old_tracker], axis=0, ignore_index=True)
print('totol tracker data ',len(ops_status_data))
print(ops_status_data.isna().sum())


totol tracker data  316948
bank_response    0
final_remarks    0
dtype: int64


In [35]:
len(raw_data)

318294

In [36]:
# Make sure everything is a string first
raw_data['sellerid'] = raw_data['sellerid'].astype(str)
raw_data['entity_id'] = raw_data['entity_id'].astype(str)
raw_data['parent_entity_id'] = raw_data['parent_entity_id'].astype(str)

# Calculate the length of each entity_id
raw_data['entity_len'] = raw_data['entity_id'].str.len()

# Extract from seller_id the last N characters based on entity_len
raw_data['seller_tail'] = raw_data.apply(lambda x: x['sellerid'][-int(x['entity_len']):], axis=1)


# Step 3: Compare both
raw_data['is_entity_match'] = raw_data['seller_tail'] == raw_data['entity_id']
raw_data['is_parent_match'] = raw_data['seller_tail'] == raw_data['parent_entity_id']

# Step 4: Create final match column
raw_data['final_match'] = raw_data['is_entity_match'] | raw_data['is_parent_match']

raw_data = raw_data[raw_data['final_match']]


len(raw_data)



277711

In [37]:
master_format[['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank']]=raw_data[['parent_id', 'mid','parent_entity_id','entity_id',
       'mid_creation_date', 'channel_date', 'cust_id', 'channel', 'edc_flag',
       'vpa_type', 'sellerid', 'bank_response', 'bank_name', 'pan_no_business',
       'pan_no_personal', 'gstin', 'gst_flag', 'category', 'sub_category',
       'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag', 'drr',
       'month_expected_gmv', 'total_child_mid', 'rank']]

len(master_format)

277711

In [38]:
gst_mapping = gst_data.set_index('solution_value')['solution_value.1'].to_dict()
master_format['GST Turn over'] = master_format['parent_id'].map(gst_mapping)

# Fill missing GSTs with 'NA'
master_format['GST Turn over'] = master_format['GST Turn over'].fillna('NA')

In [39]:
ops_status_mapping = ops_status_data.set_index('bank_response')['final_remarks'].to_dict()
master_format['OPS team status'] = master_format['bank_response'].map(ops_status_mapping)

# Fill missing GSTs with 'NA'
master_format['OPS team status'] = master_format['OPS team status'].fillna('Pending for share')

In [40]:
master_format.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1'],
      dtype='object')

In [41]:

master_format['Proposed Limit'] = np.where(
    master_format['Month_expected_GMV'].isna(), 
    master_format['limit_flag'], 
    np.maximum(np.ceil(master_format['Month_expected_GMV'] / 1e5) * 1e5, master_format['limit_flag'])
)


c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in ceil
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [42]:

# First replace NaN in 'gmv' with 0 for calculation
master_format['% usage'] = (master_format['gmv'].fillna(0) / master_format['limit_flag']) * 100




In [43]:

# Make sure % usage is numeric
master_format['% usage'] = pd.to_numeric(master_format['% usage'], errors='coerce')

# Define bins and labels
bins = [-float('inf'), 5, 15, 40, 60, 80, 100, float('inf')]
labels = ['1_<5 %', '2_5 to 15 %', '3_15-40%', '4_40-60%', '5_60-80%', '6_80-100%', '7.above 100%']

# Apply binning
master_format['Grouping'] = pd.cut(master_format['% usage'], bins=bins, labels=labels, right=False)



master_format['Grouping'].value_counts()


1_<5 %          147049
7.above 100%     47580
3_15-40%         31025
2_5 to 15 %      27218
4_40-60%         11881
5_60-80%          7874
6_80-100%         5084
Name: Grouping, dtype: int64

In [44]:

# Suppose your DataFrame is named df
liquor_categories = ['Alcoholic Beverages', 'Restaurants and Bars', 'wine and liquor shop', 'Wine producers','Alcoholic beverages']

master_format['Liquor/Others'] = master_format['sub_category'].apply(lambda x: 'Liquor' if x in liquor_categories else 'Others')


master_format['Liquor/Others'].value_counts()



Others    265598
Liquor     12113
Name: Liquor/Others, dtype: int64

In [45]:


master_format['Limit Required/Not'] = np.where(master_format['Proposed Limit'] > master_format['limit_flag'], 'Required', 'Not_required')

master_format['Limit Required/Not'].value_counts()


Not_required    219091
Required         58620
Name: Limit Required/Not, dtype: int64

In [46]:

master_format['Sum of GMV'] = master_format.groupby('bank_response')['gmv'].transform('sum')

master_format['Sum of DRR'] = master_format.groupby('bank_response')['DRR'].transform('sum')

master_format['Sum of Month Expected GMV'] = master_format.groupby('bank_response')['Month_expected_GMV'].transform('sum')

master_format['Sum of limit_flag'] = master_format.groupby('bank_response')['limit_flag'].transform('sum')

# Use numpy's vectorized operations  (MAX and ROUNDUP equivalent)
master_format['Sum of Proposed Limit'] = np.maximum(np.ceil(master_format['Sum of Month Expected GMV'] / 10**5) * 10**5, master_format['Sum of limit_flag'])

# Use numpy for the IF condition (AO2 > AN2)
master_format['Limit Required/Not.1'] = np.where(master_format['Sum of Proposed Limit'] > master_format['Sum of limit_flag'], "Required", "Not_required")

In [47]:
master_format.head(5)

,parent_id,Child MID,Parent Entity_Id,Child Entity_Id,MID Creation Date,PTY Creation Date,Cust_id,channel,EDC Flag,vpa_type,sellerid,bank_response,bank_name,Pan No Business,Pan No Personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,DRR,Month_expected_GMV,total_Child_mid,rank,Proposed Limit,% usage,Grouping,GST Turn over,Liquor/Others,Limit Required/Not,Business status,OPS team status,YBL status,Sum of GMV,Sum of DRR,Sum of Month Expected GMV,Sum of limit_flag,Sum of Proposed Limit,Limit Required/Not.1
0,wGkLWY00056042674942,wGkLWY00056042674942,84152466,84152466,11-06-2024,11-06-2024,1.899359e+09,DIY,QR,PTY,11684152466,"{""sellerId"":""11684152466""}",PTYES,CHVPG4162K,CHVPG4162K,NaN,No,Food,Juice and Snacks,5411,PROPRIETORSHIP,46255.0,400000,1595.0,49445.0,1,1,400000.0,11.56375,2_5 to 15 %,NA,Others,Not_required,NaN,Pending for share,NaN,46255.0,1595.0,49445.0,400000,400000.0,Not_required
1,wIoyDM46851761551383,wIoyDM46851761551383,87831302,87831302,21-04-2025,21-04-2025,7.043959e+06,Transit Offline,EDC,PTY,18987831302,"{""sellerId"":""18987831302""}",PTYES,CXIPS8895N,CXIPS8895N,33CXIPS8895N1Z9,Yes,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.00000,1_<5 %,Slab: Rs. 0 to 40 lakhs,Others,Not_required,NaN,ODD/EDD closed and no action required (Limit a...,NaN,0.0,0.0,0.0,750000,750000.0,Not_required
2,vmiSgy43648835767287,vmiSgy43648835767287,85538028,85538028,26-10-2024,26-10-2024,1.745045e+09,Offline Unorganized,EDC,PTY,14085538028,"{""sellerId"":""14085538028""}",PTYES,AHGPR9408N,AHGPR9408N,10AHGPR9408N1Z0,Yes,Beauty and Wellness,Salon and Parlour,7230,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.00000,1_<5 %,Slab: Rs. 0 to 40 lakhs,Others,Not_required,NaN,Uploaded,NaN,0.0,0.0,0.0,750000,750000.0,Not_required
3,vqOTUv48118460123333,vqOTUv48118460123333,84800813,84800813,27-08-2024,27-08-2024,5.104530e+08,DIY,QR,PTY,15184800813,"{""sellerId"":""15184800813""}",PTYES,JICPS9519H,JICPS9519H,NaN,No,Healthcare,Doctors and Clinic,8011,PROPRIETORSHIP,2858.0,400000,714.5,22149.5,1,1,400000.0,0.71450,1_<5 %,NA,Others,Not_required,NaN,Pending for share,NaN,2858.0,714.5,22149.5,400000,400000.0,Not_required
4,wRwpkc05430822350497,wRwpkc05430822350497,83853515,83853515,09-04-2024,07-05-2024,1.009561e+09,Offline Unorganized,QR,PTY+YSF,13183853515,"{""sellerId"":""13183853515""}",PTYES,CAKPP8317B,CAKPP8317B,NaN,No,Milk Dairy and Cooperative,Private Dairy,5451,PROPRIETORSHIP,NaN,400000,NaN,NaN,1,1,400000.0,0.00000,1_<5 %,NA,Others,Not_required,NaN,Pending for share,NaN,0.0,0.0,0.0,400000,400000.0,Not_required


In [48]:
master_format.to_csv('master_data.csv',index=False)